# Docker Swarm with Blinkt!

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [1]:
ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1

192.168.56.192


In [2]:
IP_LEADER=$(ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1)
echo $IP_LEADER

192.168.56.192


In [3]:
docker node ls || docker swarm init --advertise-addr ${IP_LEADER}

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
aodrkngn1o1qwx6qwxktf2get *   cluster-00          Ready               Active              Leader


In [4]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

SWMTKN-1-3d30vcju2ge6ee8lctlt6mvxk6obz1vcxqkrr2s6h1x7ugtzux-09fgtib58gg6d48b7w6zgd2li


In [5]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

/home/pi/.ssh/id_rsa


Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [6]:
ping -c 1 cluster-01.local | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1

192.168.56.189


In [10]:
for i in 01 02 03 04; do
    # Remove host key for ip
    CL_HOSTNAME=cluster-${i}
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1)
    echo ${CL_HOSTNAME}
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster FQDN
    ssh-keygen -R ${CL_HOSTNAME}.local
    # IP Cluster 
    ssh-keygen -R ${CL_IP}
    # Add ip into known_hosts
    ssh-keyscan -H ${CL_HOSTNAME}.local >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_HOSTNAME} >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_IP} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@${CL_HOSTNAME}.local
done

cluster-01
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host cluster-01.local found: line 18 type RSA
# Host cluster-01.local found: line 19 type ECDSA
# Host cluster-01.local found: line 20 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host 192.168.56.189 found: line 18 type ECDSA
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 192.168.56.189 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 192.168.56.189 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 192.168.56.189 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3

Alle Clients als Worker dem Swarm beitreten.

In [13]:
for i in 01 02 03 04; do
    CL_HOSTNAME=cluster-${i}.local
    echo $CL_HOSTNAME
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1)
    echo $CL_IP
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP_LEADER}:2377"
done

cluster-01.local
192.168.56.189
This node joined a swarm as a worker.
cluster-02.local
192.168.56.191
This node joined a swarm as a worker.
cluster-03.local
192.168.56.187
This node joined a swarm as a worker.
cluster-04.local
192.168.56.196
This node joined a swarm as a worker.


In [14]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
aodrkngn1o1qwx6qwxktf2get *   cluster-00          Ready               Active              Leader
c3d7jhikibvqv941bv40b0667     cluster-03          Ready               Active              
i3zg4fox6moy47tni8ho813qf     cluster-02          Ready               Active              
lb9auf6wu70b3grnlzn1ndyf9     cluster-04          Ready               Active              
yvc4npzgl2l6xa12p0oomixiu     cluster-01          Ready               Active              


## Visualizer anschauen

Einen graphischen Output als Service starten.

In [15]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  alexellis2/visualizer-arm

y3slvy3ca5vve971rqgmt4dd2
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [16]:
echo "http://${IP_LEADER}:8000"

http://192.168.56.192:8000


In [17]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
y3slvy3ca5vv        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [18]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  stefanscherer/monitor:1.1.0

7r4k5cnqvrlzupezfs09de276
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


Es läuft noch kein whoami.

In [19]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
7r4k5cnqvrlz        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
y3slvy3ca5vv        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [20]:
docker service create --name whoami stefanscherer/whoami:1.1.0

wc628t8z3haj4tel8ho6hrtjx
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [21]:
sleep 10

In [22]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
7r4k5cnqvrlz        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
wc628t8z3haj        whoami              replicated          1/1                 stefanscherer/whoami:1.1.0         
y3slvy3ca5vv        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [23]:
docker service scale whoami=5

whoami scaled to 5


In [24]:
sleep 30

In [25]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
7r4k5cnqvrlz        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
wc628t8z3haj        whoami              replicated          5/5                 stefanscherer/whoami:1.1.0         
y3slvy3ca5vv        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [26]:
docker service scale whoami=15

whoami scaled to 15


In [27]:
sleep 30

In [28]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
7r4k5cnqvrlz        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
wc628t8z3haj        whoami              replicated          15/15               stefanscherer/whoami:1.1.0         
y3slvy3ca5vv        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [29]:
docker service scale whoami=40

whoami scaled to 40


In [30]:
sleep 45

In [31]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [ ]:
sleep 45

In [ ]:
docker service ls

In [ ]:
docker service scale whoami=1

In [ ]:
sleep 15

In [ ]:
docker service scale whoami=40

In [ ]:
sleep 45

In [ ]:
docker service ls

In [ ]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.1.0 whoami

In [ ]:
sleep 45

In [ ]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

In [ ]:
sleep 45

In [ ]:
docker service ls

In [ ]:
END=10
for i in $(seq 1 $END); do
    docker service scale whoami=1 
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service scale whoami=15
    sleep 10
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 15
    docker service scale whoami=40
    sleep 15
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 30
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 30
done

# Aufräumen

In [ ]:
docker service rm whoami

In [ ]:
docker service rm monitor

In [ ]:
docker node ls

In [ ]:
docker service ls

In [ ]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

In [ ]:
for i in 21.76 23.27 23.237 23.242; do
    PRE_FIX=172.16.
    CL_HOSTNAME=${PRE_FIX}${i}
    ssh pi@${PRE_FIX}${i} "docker swarm leave --force"
done

In [ ]:
docker service rm visualizer

In [ ]:
docker swarm leave --force